# Pauper Format Price Analysis

First, get and clean the scryfall data for the format

In [1]:
import pandas as pd
import json
import requests

In [2]:
#api call and cleaning
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
oracle_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "oracle_cards"').index[0]]

response = requests.get(oracle_uri)
raw_oracle_cards = pd.json_normalize(response.json())

oracle_cards = raw_oracle_cards[[#'object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id',
       #'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 
       'name', 
       #'lang',
       #'released_at', 'uri', 'scryfall_uri', 
       'layout', 
       #'highres_image',
       #'image_status', 'mana_cost', 'cmc', 'type_line', 'oracle_text',
       #'colors', 'color_identity', 'keywords', 'games', 'reserved',
       #'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint',
       #'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri',
       #'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       #'prints_search_uri', 'collector_number', 'digital', 
       #'rarity',
       #'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       #'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       #'booster', 'story_spotlight', 'edhrec_rank', 'image_uris.small',
       #'image_uris.normal', 'image_uris.large', 'image_uris.png',
       #'image_uris.art_crop', 'image_uris.border_crop',
       #'legalities.standard',
       #'legalities.future', 'legalities.historic',
       #'legalities.gladiator', 
       #'legalities.pioneer',
       #'legalities.explorer', 
       #'legalities.modern', 
       #'legalities.legacy',
       'legalities.pauper', 
       #'legalities.vintage', 'legalities.penny',
       #'legalities.commander', 'legalities.brawl',
       #'legalities.historicbrawl', 'legalities.alchemy',
       #'legalities.paupercommander', 'legalities.duel',
       #'legalities.oldschool', 'legalities.premodern', 
       'prices.usd',
       #'prices.usd_foil', 'prices.usd_etched', 'prices.eur',
       #'prices.eur_foil', 
       'prices.tix', 
       #'related_uris.gatherer',
       #'related_uris.tcgplayer_infinite_articles',
       #'related_uris.tcgplayer_infinite_decks', 'related_uris.edhrec',
       #'security_stamp', 'preview.source', 'preview.source_uri',
       #'preview.previewed_at', 'power', 'toughness', 'penny_rank',
       #'arena_id', 'watermark', 'produced_mana', 'all_parts',
       'card_faces', 
       #'frame_effects', 'tcgplayer_etched_id',
       #'promo_types', 'loyalty', 'life_modifier', 'hand_modifier',
       #'attraction_lights', 'color_indicator', 'content_warning'
       ]]
oracle_cards = oracle_cards.rename(columns = {
    'name':'Name',
    'rarity':'Rarity',
    'legalities.pauper':'Pauper_Legal',
    'prices.usd':'Price_USD',
    'prices.tix':'Price_Tix'
})
pauper_raw = oracle_cards.query("Pauper_Legal == 'legal'").sort_values(by = ['Name']).reset_index(drop = True)

#fix multi-faced cards to match mtggoldfish naming conventions (dual faced cards, split cards)
card_faces = pd.json_normalize(pauper_raw['card_faces'].loc[~pauper_raw['card_faces'].isna()])
front_face = pd.json_normalize(card_faces[0])
back_face = pd.json_normalize(card_faces[1])

faces_index = 0 # we lose the relationship between the indices when we normalize the faces data, I'm not sure how to fix this with pandas but just keeping track of it like this works fine
for card_index in pauper_raw.loc[~pauper_raw['card_faces'].isna()].index:
    if pauper_raw['layout'].loc[card_index] == 'split':
        pauper_raw['Name'].loc[card_index] = front_face['name'].loc[faces_index]+'/'+back_face['name'].loc[faces_index]
    else:
        pauper_raw["Name"].loc[card_index] = front_face['name'].loc[faces_index]
    faces_index += 1
pauper = pauper_raw.drop(['card_faces','layout'],axis=1)
pauper.head(10)

,Name,Pauper_Legal,Price_USD,Price_Tix
0,+2 Mace,legal,0.02,0.02
1,A Good Day to Pie,legal,0.02,None
2,Abandon the Post,legal,0.01,0.03
3,Abandoned Outpost,legal,0.15,0.11
4,Abbey Griffin,legal,0.05,0.03
5,Abbey Matron,legal,0.11,None
6,Abjure,legal,0.48,0.03
7,Abnormal Endurance,legal,0.13,0.04
8,Abomination of Gudul,legal,0.11,0.03
9,Aboshan's Desire,legal,0.22,0.04


Make and fill the dataframe for pauper decklists

In [6]:
decks = pd.DataFrame(columns=['Deck_Name','Deck_List','Paper_Price','MTGO_Price_Tix'])
pauper_deck_names = ['Burn','Affinity','Dimir Control','Mono-Blue Faeries','Orzhov Ephemerate','Gruul Ponza','Familiars',
         'Boros Synthesizer','Mono-Red Synthesizer','Walls Combo','Tron','Caw Gates','4 Color Ephemerate','Ephemerate Tron',
         'Bogles','Kiln Fiend','Boros Bully','Izzet Faeries','Goblins','Slivers','Jeskai Ephemerate']

decklists = []
for deckname in pauper_deck_names:
    decklists.append(pd.read_csv('Deck Lists/Pauper Metagame Decks/'+deckname+'.csv'))

decks['Deck_Name'] = pauper_deck_names
decks['Deck_List'] = decklists

decks.head(len(decks))

,Deck_Name,Deck_List,Paper_Price,MTGO_Price_Tix
0,Burn,Quantity Card_Name 0 ...,NaN,NaN
1,Affinity,Quantity Card_Name 0 3...,NaN,NaN
2,Dimir Control,Quantity Card_Name 0 ...,NaN,NaN
3,Mono-Blue Faeries,Quantity Card_Name 0 ...,NaN,NaN
4,Orzhov Ephemerate,Quantity Card_Name 0 2...,NaN,NaN
5,Gruul Ponza,Quantity Card_Name 0 4...,NaN,NaN
6,Familiars,Quantity Card_Name 0 ...,NaN,NaN
7,Boros Synthesizer,Quantity Card_Name 0 ...,NaN,NaN
8,Mono-Red Synthesizer,Quantity Card_Name 0 ...,NaN,NaN
9,Walls Combo,Quantity Card_Name 0 ...,NaN,NaN


In [8]:
decks['Deck_List'].loc[0].head(20)

,Quantity,Card_Name
0,4,Chain Lightning
1,4,Experimental Synthesizer
2,1,Fireblast
3,4,Galvanic Blast
4,4,Goblin Blast-Runner
5,4,Great Furnace
6,4,Implement of Combustion
7,4,Kuldotha Rebirth
8,2,Lava Dart
9,4,Lightning Bolt
